In [91]:
from gpt import GPT2
from transformers import AutoTokenizer
import torch
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


In [5]:
# model = GPT2()
model = GPT2.load_weights_from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

Processing key='lm_head.weight': 100%|██████████| 149/149 [00:09<00:00, 15.96it/s]                     


## Generate Random output after loading weights

In [16]:
input = "Hello i am Vasileios Mpletsos an Electrical & Computer Engineer working on AI and ML "
encoded_text = tokenizer.encode(input, return_tensors="pt")

In [17]:
num_return_sequences = 5
max_length = 30
model.eval()
model.to("cuda");

In [18]:
encoded_text = encoded_text.repeat(num_return_sequences,1)
encoded_text = encoded_text.to("cuda")

In [19]:
while encoded_text.size(1) < max_length:
    with torch.no_grad():
        logits = model(encoded_text)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        
        topk_probs, topk_indices = torch.topk(probs, 20, dim=-1)
        selected_tokens = torch.multinomial(topk_probs,1)
        selected_tokens = torch.gather(topk_indices,-1,selected_tokens)
        encoded_text = torch.cat([encoded_text, selected_tokens], dim=1)

In [20]:
encoded_text.shape

torch.Size([5, 30])

In [21]:
for i,prediction in enumerate(encoded_text):
    decoded_generation = tokenizer.decode(prediction).strip()
    print(f"Option {i+1}: {decoded_generation}")

Option 1: Hello i am Vasileios Mpletsos an Electrical & Computer Engineer working on AI and ML  inflamm LionsGY� GreenpeaceFAQcemberphys devise Plains
Option 2: Hello i am Vasileios Mpletsos an Electrical & Computer Engineer working on AI and ML !?" Dhabi woo sacrificed Canterbury Coinbase abrelevant politely videog
Option 3: Hello i am Vasileios Mpletsos an Electrical & Computer Engineer working on AI and ML  Putin facets Gron PCIe borders Burgess exploitation caramel Ladies reborn
Option 4: Hello i am Vasileios Mpletsos an Electrical & Computer Engineer working on AI and ML  PutinPLIC specializing Sauce medi trained conservation assail invent reborn
Option 5: Hello i am Vasileios Mpletsos an Electrical & Computer Engineer working on AI and ML  layer GPUsidered seizeFX nut colleges Laksh female acclaim


## Check training loop

In [6]:
with open("./tiny_shakespeare.txt", "r") as f:
    text = f.read()

In [7]:
print(text[:400])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


In [41]:
tokens = tokenizer.encode(text, return_tensors = "pt")

In [42]:
keep_tokens = 1024*330 + 1
tokens = tokens[:,:keep_tokens]

In [57]:
input = tokens[:, :-1].view(-1,1024)
targets = tokens[:, 1:].view(-1,1024)

In [70]:
predictions = model(input[:1,:])

In [81]:
F.cross_entropy(predictions.view(-1, 50257), targets[0].view(-1))

tensor(4.2711, grad_fn=<NllLossBackward0>)

In [83]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx):
        input_tensors = self.inputs[idx]
        target_tensors = self.targets[idx]
        return input_tensors, target_tensors

In [86]:
dataset = CustomDataset(input, targets)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [90]:
next(iter(dataloader))[0].shape

torch.Size([2, 1024])

In [95]:
X_train, X_test, y_train, y_test = train_test_split(input, targets, test_size=0.2)

In [101]:
X_train.shape

torch.Size([264, 1024])